# Determining ERA5 climatology

## ERA5 climatology for deepsoiltemperture CCLM
Problem:
T_Cl in the extpar of cclm is (presumably) the reason for the cold bias in all cclm runs.
Therefore, we are investigating how other datasets could contribute to a smaller cold bias.

==> Current to do:
- Determine climatology of ERA5 soil layer 4
- Average, mean, max over 30 year periods
- Comparison with GLOBECOVER extpar and ECOCLIMAP extpar

In [1]:
import sys
parent_directory = '/dodrio/scratch/projects/2022_200/RCS/CORDEXBE2/fiens/ValEnsPy/evaluation_cclm/'
sys.path.append(parent_directory)
functions_directory = '/dodrio/scratch/projects/2022_200/RCS/CORDEXBE2/fiens/ValEnsPy/src/valenspy/'
sys.path.append(functions_directory)

from pathlib import Path

import xarray as xr
import pandas as pd

import valenspy as vp #The Valenspy package
from valenspy.inputconverter_functions import ERA5_to_CF, CCLM_to_CF
from valenspy._utilities import load_xarray_from_data_sources
from diagnostic_visualizations import *
from eval_functions import *
from yaml import safe_load

import numpy as np
import matplotlib.pyplot as plt

In [2]:
def geo_to_rot(coord, ds):
    ###
    # Converts a geographic (lon,lat) point to an (rlon,rlat) point
    # => coord: geographic lon - lat couple (degrees) => list or tuple
    # => ds: xarray dataset of a CCLM file
    ###
    # Read in COSMO-CLM data

    rp_lat = float(ds.rotated_pole.grid_north_pole_latitude)
    rp_lon = float(ds.rotated_pole.grid_north_pole_longitude)
    # Convert 
    co = np.deg2rad(coord)
    rp_lat = np.deg2rad(rp_lat); rp_lon = np.deg2rad(rp_lon)
    p_rlat = np.arcsin(np.sin(co[1])*np.sin(rp_lat) + np.cos(co[1])*np.cos(rp_lat)*np.cos(co[0]-rp_lon)) 
    p_rlon = np.arctan((np.cos(co[1])*np.sin(co[0]-rp_lon)) / (np.cos(co[1])*np.sin(rp_lat)*np.cos(co[0]-rp_lon) - np.sin(co[1])*np.cos(rp_lat))) 
    p_rlat = np.rad2deg(p_rlat); p_rlon = np.rad2deg(p_rlon)
    # Return rlon-rlat couple
    return [p_rlon,p_rlat]

In [3]:
machine = "hortense"
variable = "stl4"
ref_dataset = "ERA5"
startyear = 1961
endyear = 1990

In [4]:
# Loading ECOCLIMAP data
ec_dir = '/dodrio/scratch/projects/2022_200/RCS/CORDEXBE2/wouterl/exp_fiens_eval_setup_JJA1995_transient_aerosol_ECOCLIMAP/ext/'
ec_file = ec_dir + 'extpar_1995.nc'
ec_extp = xr.open_mfdataset(ec_file, combine="by_coords", chunks="auto")


In [5]:
# Loading ECOCLIMAP data
gc_dir = '/dodrio/scratch/projects/2022_200/RCS/CORDEXBE2/wouterl/exp_fiens_eval_setup_JJA1995_transient_aerosol/ext/'
gc_file = gc_dir + 'extpar_1995.nc'
gc_extp = xr.open_mfdataset(gc_file, combine="by_coords", chunks="auto")

In [6]:
# ERA5 files (manually loaded and converted)
data_dir = '/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/hourly/soil_temperature_level_4/'
ERA5_deepsoil_files = os.listdir(data_dir)


In [7]:
# Processing data into file with averages
# Filtering climatology files based on year range
climatology_files = [
    s for s in ERA5_deepsoil_files 
    if startyear <= int(s[-7:-3]) <= endyear
]
#climatology_files = ','.join(climatology_files)


In [8]:
# Calculating climatology statistics
# List all NetCDF files in the directory
netcdf_files = [os.path.join(data_dir, f) for f in climatology_files]
# Open multiple NetCDF files as a single xarray dataset
ds = xr.open_mfdataset(netcdf_files, combine='by_coords', engine='netcdf4', chunks="auto")



In [9]:
ds

<xarray.Dataset> Size: 50GB
Dimensions:     (valid_time: 262968, latitude: 163, longitude: 289)
Coordinates:
    number      int64 8B 0
  * valid_time  (valid_time) datetime64[ns] 2MB 1961-01-01 ... 1990-12-31T23:...
  * latitude    (latitude) float64 1kB 73.5 73.25 73.0 72.75 ... 33.5 33.25 33.0
  * longitude   (longitude) float64 2kB -27.0 -26.75 -26.5 ... 44.5 44.75 45.0
    expver      (valid_time) <U4 4MB dask.array<chunksize=(8760,), meta=np.ndarray>
Data variables:
    stl4        (valid_time, latitude, longitude) float32 50GB dask.array<chunksize=(2740, 60, 190), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-08-06T18:59 GRIB to CDM+CF via cfgrib-0.9.1...

In [10]:
# define bounds 
bounds = {'europe':
                    {'lat_bounds': [70,35], 
                    'lon_bounds': [-15,40]}, 
        'belgium': 
                    {'lat_bounds': [52,49], 
                    'lon_bounds': [2,7]}}

In [11]:
region = 'europe'
lat_bounds = bounds[region]['lat_bounds']
lon_bounds = bounds[region]['lon_bounds']
ds = ds.rename({'latitude' : 'lat', 'longitude' : 'lon', 'valid_time' : 'time'})
da = ds.sel(lon=slice(lon_bounds[0],lon_bounds[1]),lat=slice(lat_bounds[0], lat_bounds[1]))



In [12]:
da

<xarray.Dataset> Size: 33GB
Dimensions:  (time: 262968, lat: 141, lon: 221)
Coordinates:
    number   int64 8B 0
  * time     (time) datetime64[ns] 2MB 1961-01-01 ... 1990-12-31T23:00:00
  * lat      (lat) float64 1kB 70.0 69.75 69.5 69.25 ... 35.75 35.5 35.25 35.0
  * lon      (lon) float64 2kB -15.0 -14.75 -14.5 -14.25 ... 39.5 39.75 40.0
    expver   (time) <U4 4MB dask.array<chunksize=(8760,), meta=np.ndarray>
Data variables:
    stl4     (time, lat, lon) float32 33GB dask.array<chunksize=(2740, 46, 142), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-08-06T18:59 GRIB to CDM+CF via cfgrib-0.9.1...

In [19]:
da = ds['stl4'].mean('time')
#plot_map(da)

In [21]:
gridfile = data_dir + climatology_files[0]
da_ec_extp_soil = ec_extp["T_CL"]
ec_remap = remap_cdo(ds = da_ec_extp_soil, target_grid = gridfile, remap_method="bil")



In [29]:
ec_remap.attrs['long_name'] = 'CRU climatology'
ec_remap.attrs['units'] = 'K'
da_ec_remap = ec_remap.to_array()

In [22]:
diff_ec = da -  ec_remap
#diff_ec['T_CL'].plot()

In [28]:
da_diff = diff_ec.to_array()
da_diff

<xarray.DataArray (variable: 1, lat: 163, lon: 289)> Size: 188kB
dask.array<stack, shape=(1, 163, 289), dtype=float32, chunksize=(1, 60, 190), chunktype=numpy.ndarray>
Coordinates:
  * lon       (lon) float64 2kB -27.0 -26.75 -26.5 -26.25 ... 44.5 44.75 45.0
  * lat       (lat) float64 1kB 73.5 73.25 73.0 72.75 ... 33.75 33.5 33.25 33.0
    number    int64 8B 0
  * variable  (variable) object 8B 'T_CL'
Attributes:
    CDI:          Climate Data Interface version 2.4.0 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Mon Aug 12 13:15:57 2024: cdo -O -s -f nc -remapbil,/dodrio...
    CDO:          Climate Data Operators version 2.4.0 (https://mpimet.mpg.de...

In [31]:
fig = plot_maps_mod_ref_diff(da_mod = da,  da_ref = da_ec_remap,  da_diff = da_diff)